In [1]:
!pip install langchain openai arxiv pymupdf ragas==0.1.9
!pip install llama-index llama-index-llms-huggingface ipywidgets
!pip install transformers -U
!pip install sentence_transformers
!pip install unstructured
!pip install pdfminer
!pip install pypdf PyPDFLoader
!pip install rapidocr-onnxruntime
!pip install langchain_chroma
!pip install pdfplumber
!pip install langchain unstructured[all-docs] pydantic lxml langchainhub
!pip install --upgrade httpx httpx-sse PyJWT
!pip install rank_bm25
!pip install dashscope

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 11.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 25.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = PyPDFLoader("soybean_konw.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(documents[:])
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

In [3]:
from langchain import PromptTemplate
qa_generate_prompt_tmpl = """\
Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge.
generate only questions based on the below query.
You are a university professor. Your task is to set only 1 question for the upcoming Chinese quiz.
Questions throughout the test should be diverse.
Questions must be written in Chinese. The expression must be concise and clear.
It should not exceed 20 Chinese characters. Words such as "这", "那", "根据", "依据" and other punctuation marks
should not be used. Abbreviations may be used for titles and professional terms.
"""
prompt = PromptTemplate(
    template=qa_generate_prompt_tmpl, 
    input_variables=["context"]
  )

print(prompt)
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.6,
    model="glm-4-0520",
    openai_api_key="661a7aa0aeb8ca129eb4647461123230.bl9w581QKpnMfBvs",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

rag_chain = (
    {"context": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

input_variables=['context'] template='Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge.\ngenerate only questions based on the below query.\nYou are a university professor. Your task is to set only 1 question for the upcoming Chinese quiz.\nQuestions throughout the test should be diverse.\nQuestions must be written in Chinese. The expression must be concise and clear.\nIt should not exceed 20 Chinese characters. Words such as "这", "那", "根据", "依据" and other punctuation marks\nshould not be used. Abbreviations may be used for titles and professional terms.\n'


/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
content = [texts[i].page_content for i in range(len(texts))]
# content

In [5]:
question_list = rag_chain.batch(content)
# question_list

In [ ]:
import pandas as pd

# 示例字典
data_dict = {
    'id': range(len(texts)),
    'question': question_list,
    'context': content
}

# 将字典转换为Pandas DataFrame
df = pd.DataFrame(data_dict)

# 打印DataFrame
print(df)

In [7]:
df.to_excel('soybean_question_500.xlsx', index=False)